# Notas varias de preparación
(celda oculta)

In [76]:
#esto lo tuve que correr en la consola
#!conda install jupyterthemes
#!jt -t chesterish

'''
# estos son los temas que están disponibles
    onedork
    grade3
    oceans16
    chesterish
    monokai
    solarizedl
    solarizedd

'''

#!apt-get -qq install -y urlli
#!python3 -m spacy download en_core_web_lg
#!pip install  textacy
#!conda install -c conda-forge spacy-model-en_core_web_lg 
#!pip uninstall  spacy
#después de instalar spacy fue necesario reinstalar jupiter en el ambiente


'\n# estos son los temas que están disponibles\n    onedork\n    grade3\n    oceans16\n    chesterish\n    monokai\n    solarizedl\n    solarizedd\n\n'

# Importación de librerias

In [77]:
# Import libraries
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import sys
#para hacer nlp
import spacy
import textacy.extract

# tuve que correr el siguiente comando para instalar la version lg (826 MB)
#!python3 -m spacy download en_core_web_lg
# sino tenía que ejecutar nlp = spacy.load('en_core_web_sm')
nlp = spacy.load('en_core_web_lg')
nlp2 = spacy.load('en_core_web_lg')
#nlp = spacy.load('en_core_web_sm')



# Carga de contenido de capítulos de Star Trek
## web scrapping y parseo

- Paso 1: obtener todos los links de los capítulos de la serie


 Ejemplos seguidos
- - https://scipython.com/blog/scraping-a-wikipedia-table-with-beautiful-soup/
- - https://towardsdatascience.com/how-to-web-scrape-with-python-in-4-minutes-bc49186a8460
- - https://www.crummy.com/software/BeautifulSoup/bs4/doc/
- no fue necesario, pero tener en cuenta urljoin(url_base ,a.get('href'))


Paso 2: obtener todos los plot de los capítulos de la serie
- esto es para tomar el contenido del plot cuando ya estoy en la página del capítulo
- como explícitamente quiero haer lenta la iteración de las páginas para asignar el plot, itero a mano por el índice (pendiente)
- Se limpia el contenido html y se deja solamente el texto usando BeautifulSoup


In [78]:

# Set the URL you want to webscrape from
url_base = 'https://en.wikipedia.org/wiki/List_of_Star_Trek:_The_Original_Series_episodes'
# Connect to the URL
response = requests.get(url_base)
# Parse HTML and save to BeautifulSoup object¶
soup = BeautifulSoup(response.text, "html.parser")
# Paso 1: obtener todos los links de los capítulos de la serie
# Ejemplos seguidos
# https://scipython.com/blog/scraping-a-wikipedia-table-with-beautiful-soup/
# https://towardsdatascience.com/how-to-web-scrape-with-python-in-4-minutes-bc49186a8460
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
# no fue necesario, pero tener en cuenta urljoin(url_base ,a.get('href'))
# BORRAR LA LÍNEA QUE CONFIGURA UN MÁXIMO DE LINKS


#guardo el nro de capitulo en la serie y el absoluto, el título del capítulo y el link correspondiente
columns_df = ['No.overall', 'No. inseason', 'Title', 'Stardate', 'Link']
chapters_link = pd.DataFrame(columns=columns_df)

#itero sobre todas las tablas de la página filtrando por class
#BORRAR LUEGO, ESTO ES PARA DESA
seasonsCount=0
maxSeasons = 2
maxChapters=6
for table in soup.findAll('table', attrs={'class':'wikitable plainrowheaders wikiepisodetable'}):
    #BORRAR LUEGO, ESTO ES PARA DESA
    if seasonsCount<maxSeasons:
      
      seasonsCount +=1
      ths = table.find_all('th')
      headings = [th.text.strip() for th in ths]
            
      # verifico si es del tipo de tabla que me interesa
      if headings[:3] == ['No.overall', 'No. inseason', 'Title']:
        #para topear la cantidad de links por temporada
        chapters_count = 0
        for row in table.tbody.findAll('tr'):
          #
          if chapters_count <= maxChapters :
            chapters_count +=1
            try:
              nooverall = row.findAll('th')[0].string
              noinseason = row.findAll('td')[0].string
              title = row.findAll('td')[1].find('a').string
              stardate = row.findAll('td')[2].string
              link = urljoin(url_base ,row.findAll('td')[1].find('a').get('href'))
              
                        
              chapters_link = chapters_link.\
                            append(\
                                    pd.DataFrame([[nooverall, noinseason, title, stardate, link]], \
                                    columns=columns_df), ignore_index=True)
            except:
                #hay filas que no tienen el contenido deseado, así que las ignoro cuando fallan
                pass
                #print(sys.exc_info())

# Paso 2: obtener todos los plot de los capítulos de la serie
# esto es para tomar el contenido del plot cuando ya estoy en la página del capítulo
# como explícitamente quiero haer lenta la iteración de las páginas para asignar el plot, itero a mano por el índice (pendiente)
plotsHtml = []
plots = []
for index, row in chapters_link.iterrows():
  chapter_page = requests.get(row[4])
  chapter_soup = BeautifulSoup(chapter_page.text, "html.parser")
  plotsHtml.append(chapter_soup.find('span', attrs={'id':'Plot'}).parent.next_sibling.next_sibling)
  plots.append(chapter_soup.find('span', attrs={'id':'Plot'}).parent.next_sibling.next_sibling.getText())
  print('guardado plot {}'.format(index))
  time.sleep(1)
chapters_link['Plot'] = plots
chapters_link['PlotHtml'] = plotsHtml
print('Fin de carga de plots')

guardado plot 0
guardado plot 1
guardado plot 2
guardado plot 3
guardado plot 4
guardado plot 5
Fin de carga de plots


In [79]:
#desa - es para poder ver qué tiene un plot y todos los plot concatenados
print('Contenido del primer capítulo: \n\n', chapters_link.Plot[0])
print('\n\nContenido de todo el dataframe:')
chapters_link

Contenido del primer capítulo: 

 The USS Enterprise arrives at planet M-113 to provide supplies and medical exams for the only known inhabitants of the planet, Professor Robert Crater (Alfred Ryder) and his wife Nancy (Jeanne Bal), who operate an archaeological research station there. Captain Kirk, Chief Medical Officer Dr. Leonard McCoy, and Crewman Darnell (Michael Zaslow) transport to the surface as Kirk teases McCoy about his affection for Nancy ten years earlier. They arrive in the research station, and each of the three men sees Nancy differently: McCoy as she was when he first met her, Kirk as she should look accounting for her age, and Darnell as an attractive blonde woman who he met on a pleasure planet. When Nancy goes out to fetch her husband, she beckons Darnell to follow her.



Contenido de todo el dataframe:


,No.overall,No. inseason,Title,Stardate,Link,Plot,PlotHtml
0,1,1,The Man Trap,1513.1,https://en.wikipedia.org/wiki/The_Man_Trap,The USS Enterprise arrives at planet M-113 to ...,"[The , [USS , [Enterprise]], arrives at plane..."
1,2,2,Charlie X,1533.6,https://en.wikipedia.org/wiki/Charlie_X,The USS Enterprise meets the merchant vessel A...,"[The , [USS , [Enterprise]], meets the mercha..."
2,3,3,Where No Man Has Gone Before,1312.4,https://en.wikipedia.org/wiki/Where_No_Man_Has...,The USS Enterprise is on an exploratory missio...,"[The , [USS , [Enterprise]], is on an explora..."
3,4,4,The Naked Time,1704.2,https://en.wikipedia.org/wiki/The_Naked_Time,The USS Enterprise under the command of Captai...,"[The , [USS , [Enterprise]], under the comman..."
4,5,5,The Enemy Within,1672.1,https://en.wikipedia.org/wiki/The_Enemy_Within...,The USS Enterprise is on a geological explorat...,"[The , [USS , [Enterprise]], is on a geologic..."
5,6,6,Mudd's Women,1329.8,https://en.wikipedia.org/wiki/Mudd%27s_Women,The USS Enterprise under the command of Captai...,"[The , [USS , [Enterprise]], under the comman..."


# Comienzo de procesamiento de los capítulos

## Revisión del primer capítulo

In [80]:
doc = nlp(chapters_link.Plot[0])
#separo el capítulo 1 en oraciones e imprimo la primera
sentence_spans=list(doc.sents)
sentence_spans[0]

The USS Enterprise arrives at planet M-113 to provide supplies and medical exams for the only known inhabitants of the planet, Professor Robert Crater (Alfred Ryder) and his wife Nancy (Jeanne Bal), who operate an archaeological research station there.

In [81]:
# Imprimo las entidades (NER; Name Entity Recognition) que Spacy identificó
spacy.displacy.render(sentence_spans[0:2], style="ent")

Notar que se pudo haber incluido el título, profesión o cargo a la persona (ej. "Dr. Leonard McCoy" en lugar de sólo  "Leonard McCoy"); para estos casos ver el capítulo "Expanding named entities" (https://spacy.io/usage/rule-based-matching#models-rules-ner)

In [82]:
# Imprimo la lista de entidades
ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print(*list(ents), sep="\n")

('The USS Enterprise', 0, 18, 'PRODUCT')
('Robert Crater', 137, 150, 'PERSON')
('Alfred Ryder', 152, 164, 'PERSON')
('Nancy', 179, 184, 'PERSON')
('Jeanne Bal', 186, 196, 'PERSON')
('Kirk', 261, 265, 'PERSON')
('Leonard McCoy', 293, 306, 'PERSON')
('Crewman Darnell', 312, 327, 'PERSON')
('Michael Zaslow', 329, 343, 'PERSON')
('Kirk', 373, 377, 'PERSON')
('McCoy', 385, 390, 'PERSON')
('Nancy', 415, 420, 'PERSON')
('ten years earlier', 421, 438, 'DATE')
('three', 493, 498, 'CARDINAL')
('Nancy', 508, 513, 'PERSON')
('McCoy', 527, 532, 'PERSON')
('first', 552, 557, 'ORDINAL')
('Kirk', 567, 571, 'PERSON')
('Darnell', 619, 626, 'PERSON')
('Nancy', 695, 700, 'PERSON')
('Darnell', 744, 751, 'PERSON')


# Objetivo: identificar acciones de personas (Persona + Verbo)

In [83]:
#backup
'''
ruler = spacy.pipeline.EntityRuler
patterns = [{"label": "PERSON"},
            {"label": "GPE", "pattern": [{"LOWER": "san"}, {"LOWER": "francisco"}]}]
ruler.add_patterns(patterns)
nlp.add_pipe(ruler)
'''


'\nruler = spacy.pipeline.EntityRuler\npatterns = [{"label": "PERSON"},\n            {"label": "GPE", "pattern": [{"LOWER": "san"}, {"LOWER": "francisco"}]}]\nruler.add_patterns(patterns)\nnlp.add_pipe(ruler)\n'

In [84]:
ruler = spacy.pipeline.EntityRuler(nlp2)
patterns = [{"label": "PERSON", "pattern": "Kirk"}]
ruler.add_patterns(patterns)
nlp2.add_pipe(ruler)

In [108]:
doc2 =  nlp2(chapters_link.Plot[0])
#print([(ent.text, ent.label_) for ent in doc2.ents])
ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc2.ents]
print(*list(ents), sep="\n")


('The USS Enterprise', 0, 18, 'PRODUCT')
('Robert Crater', 137, 150, 'PERSON')
('Alfred Ryder', 152, 164, 'PERSON')
('Nancy', 179, 184, 'PERSON')
('Jeanne Bal', 186, 196, 'PERSON')
('Kirk', 261, 265, 'PERSON')
('Leonard McCoy', 293, 306, 'PERSON')
('Crewman Darnell', 312, 327, 'PERSON')
('Michael Zaslow', 329, 343, 'PERSON')
('Kirk', 373, 377, 'PERSON')
('McCoy', 385, 390, 'PERSON')
('Nancy', 415, 420, 'PERSON')
('ten years earlier', 421, 438, 'DATE')
('three', 493, 498, 'CARDINAL')
('Nancy', 508, 513, 'PERSON')
('McCoy', 527, 532, 'PERSON')
('first', 552, 557, 'ORDINAL')
('Kirk', 567, 571, 'PERSON')
('Darnell', 619, 626, 'PERSON')
('Nancy', 695, 700, 'PERSON')
('Darnell', 744, 751, 'PERSON')
<generator object <genexpr> at 0x0000021FA4472AC8>


In [109]:
for token in doc2:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

The the DET DT det Xxx True True
USS USS PROPN NNP compound XXX True False
Enterprise Enterprise PROPN NNP nsubj Xxxxx True False
arrives arrive VERB VBZ ROOT xxxx True False
at at ADP IN prep xx True True
planet planet NOUN NN pobj xxxx True False
M-113 M-113 NOUN NNS dep X-ddd False False
to to PART TO aux xx True True
provide provide VERB VB advcl xxxx True False
supplies supply NOUN NNS dobj xxxx True False
and and CCONJ CC cc xxx True True
medical medical ADJ JJ amod xxxx True False
exams exam NOUN NNS conj xxxx True False
for for ADP IN prep xxx True True
the the DET DT det xxx True True
only only ADV RB amod xxxx True True
known know VERB VBN amod xxxx True False
inhabitants inhabitant NOUN NNS pobj xxxx True False
of of ADP IN prep xx True True
the the DET DT det xxx True True
planet planet NOUN NN pobj xxxx True False
, , PUNCT , punct , False False
Professor Professor PROPN NNP compound Xxxxx True False
Robert Robert PROPN NNP compound Xxxxx True False
Crater Crater PROPN NNP

In [97]:
print([(ent.text, ent.label_) for ent in doc2.ents])

[('The USS Enterprise', 'PRODUCT'), ('Robert Crater', 'PERSON'), ('Alfred Ryder', 'PERSON'), ('Nancy', 'PERSON'), ('Jeanne Bal', 'PERSON'), ('Kirk', 'PERSON'), ('Leonard McCoy', 'PERSON'), ('Crewman Darnell', 'PERSON'), ('Michael Zaslow', 'PERSON'), ('Kirk', 'PERSON'), ('McCoy', 'PERSON'), ('Nancy', 'PERSON'), ('ten years earlier', 'DATE'), ('three', 'CARDINAL'), ('Nancy', 'PERSON'), ('McCoy', 'PERSON'), ('first', 'ORDINAL'), ('Kirk', 'PERSON'), ('Darnell', 'PERSON'), ('Nancy', 'PERSON'), ('Darnell', 'PERSON')]


In [129]:
from spacy.lang.en import English
from spacy.pipeline import EntityRuler

nlp = English()
ruler = EntityRuler(nlp)
patterns = [{"label": "ORG", "pattern": "Apple"},
            {"label": "GPE", "pattern": [{"LOWER": "san"}, {"LOWER": "francisco"}]}]

patterns = [{"label": "PERSON", "pattern": "Kirk"}]
#ruler.add_patterns(patterns)
#nlp.add_pipe(ruler)

ruler.add_patterns(patterns)
nlp.add_pipe(ruler)

doc = nlp(chapters_link.Plot[0])
print([(ent.text, ent.start_char, ent.end_char,ent.label_, ent.root.head.lemma) for ent in doc.ents])



[('Kirk', 261, 265, 'PERSON', 7978019779053590073), ('Kirk', 373, 377, 'PERSON', 7978019779053590073), ('Kirk', 567, 571, 'PERSON', 7978019779053590073)]


In [102]:
#ver código en https://spacy.io/usage/rule-based-matching#models-rules-ner

from spacy.pipeline import merge_entities
from spacy import displacy


def extract_person_orgs(doc):
    person_entities = [ent for ent in doc.ents if ent.label_ == "PERSON"]
    for ent in person_entities:
        head = ent.root.head
        if head.lemma_ == "work":
            preps = [token for token in head.children if token.dep_ == "prep"]
            for prep in preps:
                orgs = [token for token in prep.children if token.ent_type_ == "ORG"]
                print({'person': ent, 'orgs': orgs, 'past': head.tag_ == "VBD"})
    return doc

# To make the entities easier to work with, we'll merge them into single tokens
nlp.add_pipe(merge_entities)
nlp.add_pipe(extract_person_orgs)

doc = nlp("Alex Smith worked at Acme Corp Inc.")
# If you're not in a Jupyter / IPython environment, use displacy.serve
displacy.render(doc, options={'fine_grained': True})

In [128]:

from spacy import displacy
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)
person_action_pattern= [
                    {'POS':{'IN':['PROPN']}}]
matcher.add("ACTION_PTTRN", None, person_action_pattern)
for match_id, start, end in matcher(doc):
    print(doc[start:end])

In [125]:

from spacy import displacy
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)
person_action_pattern= [{'LOWER':'kirk'}, {"POS": "ADV", "OP": "*"}]
matcher.add("ACTION_PTTRN", None, person_action_pattern)
for match_id, start, end in matcher(doc):
    print(doc[start:end])

ValueError: [E155] The pipeline needs to include a tagger in order to use Matcher or PhraseMatcher with the attributes POS, TAG, or LEMMA. Try using nlp() instead of nlp.make_doc() or list(nlp.pipe()) instead of list(nlp.tokenizer.pipe()).

In [86]:
#Guardo un plot y todo el texto de la serie para comparar
#print(chapters_link['Plot'].str.join('').shape)
doc_serie = nlp(chapters_link.Plot.str.cat())
doc_serie_merged= nlp(chapters_link.Plot.str.cat())
doc_plot1 = nlp(chapters_link['Plot'][0])
doc_plot1_merged = nlp(chapters_link['Plot'][0])

for entity in doc_serie_merged.ents:
    entity.merge()
for entity in doc_plot1_merged.ents:
    entity.merge()

In [87]:
#PENDIENTE: para comparar las entidades antes y después de hacer merge
print('sin hacer merge')
for entity in doc_serie_merged.ents:
    print(f"{entity.text} ({entity.label_})")
print('\n\nhaciendo merge')
for entity in doc_plot1_merged.ents:
    print(f"{entity.text} ({entity.label_})")

sin hacer merge
The USS Enterprise (PRODUCT)
Robert Crater (PERSON)
Alfred Ryder (PERSON)
Nancy (PERSON)
Jeanne Bal (PERSON)
Kirk (PERSON)
Leonard McCoy (PERSON)
Crewman Darnell (PERSON)
Michael Zaslow (PERSON)
Kirk (PERSON)
McCoy (PERSON)
Nancy (PERSON)
ten years earlier (DATE)
three (CARDINAL)
Nancy (PERSON)
McCoy (PERSON)
first (ORDINAL)
Kirk (PERSON)
Darnell (PERSON)
Nancy (PERSON)
Darnell (PERSON)
The USS Enterprise (PRODUCT)
Antares (PRODUCT)
Charlie Evans (PERSON)
fourteen years (DATE)
Charlie (PERSON)
Charlie (PERSON)
Alpha V.  Crew (ORG)
Antares (ORG)
Charlie (PERSON)
McCoy (PERSON)
Antares (PRODUCT)
The USS Enterprise (PRODUCT)
the SS Valiant (ORG)
Earth (LOC)
200 years earlier (DATE)
Valiant (PRODUCT)
Valiant (PRODUCT)
The USS Enterprise (PRODUCT)
Kirk (PERSON)
Spock (PERSON)
Lt (PERSON)
Joe Tormolen (PERSON)
Tormolen (PERSON)
Dr. McCoy (PERSON)
McCoy (PERSON)
The USS Enterprise (ORG)
177 (CARDINAL)
Geological Technician Fisher (ORG)
Scott (PERSON)
Fisher (PERSON)
The USS En

## Comparación de Tokens con y sin Merge
En esta comparación se nota el efecto de hacer merge y luego hacer la verificación de frecuencia de palabras.
Sin hacer Merge, USS y Enterprise son dos tokens distintos cuando se está analizando el Plot.

Notar que se encuentran casos donde "Enterprise" aparece sin el "USS" y por ende aparece como token por separado.

### Ver cómo se puede hacer para considerarlos iguales (sinónimos), porque lo mismo pasa con 'Kirk' y 'Captain Kirk'

In [88]:
from collections import Counter

docs = [doc_serie, doc_serie_merged, doc_plot1, doc_plot1_merged]
for doc_actual in docs:
    words = [token.text for token in doc_actual \
            if not token.is_stop and not token.is_punct]
    word_freq = Counter(words)
    common_words = word_freq.most_common(10)
    print(f"\nsalida \n{common_words}")


salida 
[('Enterprise', 10), ('planet', 8), ('ship', 7), ('USS', 6), ('Kirk', 6), ('McCoy', 6), ('\n', 6), ('Nancy', 4), ('Charlie', 4), (' ', 4)]

salida 
[('planet', 8), ('ship', 7), ('The USS Enterprise', 6), ('Kirk', 6), ('\n', 6), ('Nancy', 4), ('McCoy', 4), ('Enterprise', 4), ('research', 3), ('Captain', 3)]

salida 
[('Nancy', 4), ('planet', 3), ('Kirk', 3), ('McCoy', 3), ('Darnell', 3), ('research', 2), ('station', 2), ('met', 2), ('USS', 1), ('Enterprise', 1)]

salida 
[('Nancy', 4), ('planet', 3), ('Kirk', 3), ('research', 2), ('station', 2), ('McCoy', 2), ('met', 2), ('Darnell', 2), ('The USS Enterprise', 1), ('arrives', 1)]


## Búsqueda de sinónimos o nombres compuestos

In [89]:
print(f"Enterprise vs USS Enterprise {nlp('Enterprise').similarity(nlp('USS Enterprise'))}")
print(f"Kirk vs Captain Kirk {nlp('Kirk').similarity(nlp('Captain Kirk'))}")
print(f"Captain vs Captain Kirk {nlp('Captain').similarity(nlp('Captain Kirk'))}")
print(f"Pol vs Captain Pol {nlp('Pol').similarity(nlp('Captain Pol'))}")
print(f"Captain vs Captain Pol {nlp('Captain').similarity(nlp('Captain Pol'))}")
print(f"Captain Kirk vs Captain Pol {nlp('Captain Kirk').similarity(nlp('Captain Pol'))}")

Enterprise vs USS Enterprise 0.7250052656795019
Kirk vs Captain Kirk 0.7703250716419314
Captain vs Captain Kirk 0.8261662271499984
Pol vs Captain Pol 0.6360505226281187
Captain vs Captain Pol 0.7397638974070903
Captain Kirk vs Captain Pol 0.6690012729041105


In [90]:
doc = doc_serie
statements = textacy.extract.semistructured_statements(doc,"Darnell")
# Prints Results
print("This text is about: ")
for statement in statements:
    subject,verb,point = statement
    print(f':{point}')

This text is about: 


In [91]:
#este no anda
statements = textacy.extract.semistructured_statements(
    doc,
    "Kirk"
)
print('Facts about Kirk :')
for statement in statements:
  subject, verb, fact = statement
  print(f" - {subject}")

Facts about Kirk :


In [92]:
textacy.extract.ngrams(doc, 3, filter_stops=True, filter_punct=True, filter_nums=False, include_pos=None, exclude_pos=None, min_freq=3)

<generator object ngrams at 0x0000021FA4472DC8>

In [93]:

sentences = textacy.extract.semistructured_statements(doc, 'The USS Enterprise', cue = 'be')
for i, x in enumerate(sentences):
        subject, verb, fact = x
        
        print( 'Fact '+str(i+1) +': '+(str(fact))+" ")

Fact 1: on an exploratory mission to leave the galaxy 
Fact 2: on a geological exploration of the planet Alpha 177 
Fact 3: in pursuit of an unregistered cargo spaceship 


In [94]:
noun_chunks = textacy.extract.noun_chunks(
    doc,
    min_freq=1
)
noun_chunks = map(str, noun_chunks)
noun_chunks = map(str.lower, noun_chunks)

for noun_chunk in set(noun_chunks):
  if len(noun_chunk.split(" "))>1 :
    print(noun_chunk)

dying planet
environmental suit glove
only known inhabitants
escape attempt
sole survivor
self-destruct order
kirk orders enterprise's shields
research team
fisher's uniform
medical issues
attractive blonde woman
geological technician fisher
research station
mr. spock
asteroid field
landing party
ss valiant
extrasensory perception
dr. leonard mccoy
other spacecraft
magnetic dust
pleasure planet
charlie evans
joe tormolen
medical exams
three men
planet thasus
unregistered cargo spaceship
lithium crystal circuits
researchers' life support system
chief engineer scott
strange red liquid
alpha v.  crew members
fourteen years
exploratory mission
damaged ship's recorder
transporter equipment
chief medical officer
planet's breakup
ship's computer systems
control console
enterprise's warp engines.[note
nearest relatives
michael zaslow
captain kirk
geological exploration
alfred ryder
earth spaceship
uss enterprise
ship's library computer
jeanne bal
transport ship
dr. mccoy
"magnetic space storm


In [95]:
# TO DO LIST
# Lista de actores para quitar del análisis del texto
# GRAFICAR LÍNEA DE TIEMPO CON CAPITULOS
# BAG OF WORDS DE CAPÍTULOS
# BAG OF WORDS DE TEMPORADAS
# BAG OF WORDS DE SERIE COMPLETA
# LISTA DE PERSONAJES POR CAPITULO, TEMPORADA Y SERIE
